In [97]:
# import library
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout, Input, Concatenate
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [98]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
filePath = '/content/drive/MyDrive/riset-gamelan/ganLengkap.xlsx'

In [100]:
# filePath = 'ganLengkap.xlsx'
dataTrain = pd.read_excel(filePath, sheet_name='databinerSemua')
dataTesting = pd.read_excel(filePath, sheet_name='dataTesting')

In [101]:
def readData(df, start, jumlahData):
    columns = df[2:, start:]
    return [columns[:, i:i+1] for i in range(0, jumlahData)]

In [102]:
# dummyData = dataTrain.to_numpy()

# X_Note1 = dummyData[:7, 8:9]
# X_Note4 = dummyData[:7, 11:12]

In [103]:
# print(X_Note1, X_Note4)

In [104]:
def getData(df):
    dataDict = {}
    # data ganLengkap.xlsx
    ranges = {
        'X': (1, 11),
        'Y_kenong': (12, 8),
        'Y_kethuk': (20, 8),
        'Y_kempul': (28, 4),
        'Y_gong': (32, 2), # terdiri dari gong kecil dan gong besar
        'Y_kempyang': (34, 8),
        'Y_bb': (42, 32),
        'Y_bp': (74, 64),
        'Y_peking': (138, 32)
    }

    for name, (start, jumlahData) in ranges.items():
        dataDict[name] = readData(df, start, jumlahData)

    return dataDict

In [105]:
def encodeOneHot(data):
    encoder = LabelEncoder()
    data = data.ravel()
    encodedData = encoder.fit_transform(data)
    oneHotLabels = to_categorical(encodedData)
    return oneHotLabels, encoder

In [106]:
def getEncodeOneHot(data_dict):
    encodedData = {}
    encoders = {}

    keys = {
        "X": 11,
        "Y_kenong": 8,
        "Y_kethuk": 8,
        "Y_kempul": 4, # dataGamelan.xlsx 5
        "Y_gong": 2,
        "Y_kempyang": 8,
        "Y_bb": 32,
        "Y_bp": 64,
        "Y_peking": 32
    }

    # data yang harus diubah ke dalam string
    convertToStr = {
        "X": [7, 8, 9, 10],
        "Y_bb": list(range(32)),
        "Y_bp": list(range(64)),
        "Y_peking": list(range(32))
    }

    for key, count in keys.items():
        encodedData[key] = []
        encoders[key] = []

        for i in range(count):
            data = data_dict[key][i]

            # convert ke string
            if key in convertToStr and i in convertToStr[key]:
                data = data.astype('str')

            oneHotEncoded, encoder = encodeOneHot(data)
            encodedData[key].append(oneHotEncoded)
            encoders[key].append(encoder)

    return encodedData, encoders

In [107]:
dataTrain= dataTrain.to_numpy()
dataDictTrain = getData(dataTrain)
encodedDataTrain, encodersTrain = getEncodeOneHot(dataDictTrain)

dataTesting= dataTesting.to_numpy()
dataDictTest = getData(dataTesting)
encodedDataTest, encodersTest = getEncodeOneHot(dataDictTest)

In [108]:
# contoh data X Note 4
dataDictTrain["X"][10][:7]

array([[5],
       [3],
       [3],
       [5],
       [2],
       [6],
       [5]], dtype=object)

In [109]:
# contoh data one hot X Note 4
encodedDataTrain["X"][10][:7]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [110]:
# kenong
outputTrainKenong1 = encodedDataTrain["Y_kenong"][0].shape[1]
outputTrainKenong2 = encodedDataTrain["Y_kenong"][1].shape[1]
outputTrainKenong3 = encodedDataTrain["Y_kenong"][2].shape[1]
outputTrainKenong4 = encodedDataTrain["Y_kenong"][3].shape[1]
outputTrainKenong5 = encodedDataTrain["Y_kenong"][4].shape[1]
outputTrainKenong6 = encodedDataTrain["Y_kenong"][5].shape[1]
outputTrainKenong7 = encodedDataTrain["Y_kenong"][6].shape[1]
outputTrainKenong8 = encodedDataTrain["Y_kenong"][7].shape[1]

# kethuk
outputTrainKethuk1 = encodedDataTrain["Y_kethuk"][0].shape[1]
outputTrainKethuk2 = encodedDataTrain["Y_kethuk"][1].shape[1]
outputTrainKethuk3 = encodedDataTrain["Y_kethuk"][2].shape[1]
outputTrainKethuk4 = encodedDataTrain["Y_kethuk"][3].shape[1]
outputTrainKethuk5 = encodedDataTrain["Y_kethuk"][4].shape[1]
outputTrainKethuk6 = encodedDataTrain["Y_kethuk"][5].shape[1]
outputTrainKethuk7 = encodedDataTrain["Y_kethuk"][6].shape[1]
outputTrainKethuk8 = encodedDataTrain["Y_kethuk"][7].shape[1]


# kempul
outputTrainKempul1 = encodedDataTrain["Y_kempul"][0].shape[1]
outputTrainKempul2 = encodedDataTrain["Y_kempul"][1].shape[1]
outputTrainKempul3 = encodedDataTrain["Y_kempul"][2].shape[1]
outputTrainKempul4 = encodedDataTrain["Y_kempul"][3].shape[1]
# outputTrainKempul5 = encodedDataTrain["Y_kempul"][4].shape[1]

# gong
outputTrainGK = encodedDataTrain["Y_gong"][0].shape[1]
outputTrainGB = encodedDataTrain["Y_gong"][1].shape[1]

# kempyang
outputTrainKempyang1 = encodedDataTrain["Y_kempyang"][0].shape[1]
outputTrainKempyang2 = encodedDataTrain["Y_kempyang"][1].shape[1]
outputTrainKempyang3 = encodedDataTrain["Y_kempyang"][2].shape[1]
outputTrainKempyang4 = encodedDataTrain["Y_kempyang"][3].shape[1]
outputTrainKempyang5 = encodedDataTrain["Y_kempyang"][4].shape[1]
outputTrainKempyang6 = encodedDataTrain["Y_kempyang"][5].shape[1]
outputTrainKempyang7 = encodedDataTrain["Y_kempyang"][6].shape[1]
outputTrainKempyang8 = encodedDataTrain["Y_kempyang"][7].shape[1]

In [111]:
# outputTrain bonang barung
outputTrainBarung1 = encodedDataTrain["Y_bb"][0].shape[1]
# ....
outputTrainBarung32 = encodedDataTrain["Y_bb"][31].shape[1]

In [112]:
# outputTrain bonang penerus
outputTrainPenerus1 = encodedDataTrain["Y_bp"][0].shape[1]
# ...
outputTrainPenerus64 = encodedDataTrain["Y_bp"][63].shape[1]

In [113]:
# outputTrain peking
outputTrainPeking1 = encodedDataTrain["Y_peking"][0].shape[1]
# ...
outputTrainPeking32 = encodedDataTrain["Y_peking"][0].shape[1]

In [114]:
def buildGenerator(latent_dim, conditional_dim, output_dim):
    latent_input = Input(shape=(latent_dim,))
    conditional_input = Input(shape=(conditional_dim,))
    merged_input = Concatenate()([latent_input, conditional_input])

    x = Dense(128)(merged_input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dense(512)(x)
    x = LeakyReLU(alpha=0.2)(x)
    output = Dense(output_dim, activation='tanh')(x)

    model = Model([latent_input, conditional_input], output)
    return model

In [115]:
def buildDiscriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_shape=(input_dim,)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [116]:
def buildGan(generator, discriminator):
    latent_input = Input(shape=(latent_dim,))
    conditional_input = Input(shape=(conditional_dim,))
    generator_output = generator([latent_input, conditional_input])
    discriminator_output = discriminator(generator_output)
    model = Model([latent_input, conditional_input], discriminator_output)
    return model


In [117]:
def trainGan(gan, generator, discriminator, X, Y, latent_dim, epochs=1000, batch_size=64):
    half_batch = int(batch_size / 2)

    d_losses_real = []
    d_losses_fake = []
    g_losses = []

    for epoch in range(epochs):
        # Mini-batch di epoch
        for _ in range(int(X.shape[0] / batch_size)):
            # Train discriminator real data
            idx = np.random.randint(0, X.shape[0], half_batch)
            real_X = X[idx]
            real_Y = Y[idx]

            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_Y = generator.predict([noise, real_X])

            # Train real data
            d_loss_real = discriminator.train_on_batch(real_Y, np.ones((half_batch, 1)))
            # Train fake data
            d_loss_fake = discriminator.train_on_batch(fake_Y, np.zeros((half_batch, 1)))

            d_losses_real.append(d_loss_real)
            d_losses_fake.append(d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            sampled_X = X[np.random.randint(0, X.shape[0], batch_size)]
            valid_y = np.ones((batch_size, 1))

            g_loss = gan.train_on_batch([noise, sampled_X], valid_y)
            g_losses.append(g_loss)

        if epoch % 100 == 0:
            print(f"{epoch} [D loss real: {d_loss_real[0]}, D loss fake: {d_loss_fake[0]}, acc real: {100 * d_loss_real[1]}%, acc fake: {100 * d_loss_fake[1]}%] [G loss: {g_loss}]")

    return d_losses_real, d_losses_fake, g_losses


In [118]:
def inverseOneHot(one_hot_encoded, encoder):
    # Mengambil indeks kelas dari one-hot encoding
    index = np.argmax(one_hot_encoded)
    # Mengembalikan nilai asli dari indeks tersebut
    originalValue = encoder.inverse_transform([index])
    return originalValue[0]

def decodeData(encoded_data, encoders, instrument_indices, indexAt=None):
    startIndeks = 0
    labelsDecoded = {}

    if indexAt is not None:
        encoded_data = encoded_data[indexAt]

    for instrument, count in instrument_indices.items():
        labelsDecoded[instrument] = []
        for i in range(count):
            endIndeks = startIndeks + len(encoders[instrument][i].classes_)
            oneHotPart = encoded_data[startIndeks:endIndeks]

            # Decode the one-hot encoded part into the original value
            originalValue = inverseOneHot(oneHotPart, encoders[instrument][i])
            labelsDecoded[instrument].append(originalValue)

            startIndeks = endIndeks  # Update start index for the next instrument

    return labelsDecoded


In [119]:
# def accuracyGan(real_data, generated_data): return sum(values[i] == generated_data[key][i] for key, values in real_data.items() for i in range(len(values))) / sum(len(values) for values in real_data.values())

In [120]:
XTrain = np.concatenate((encodedDataTrain["X"][0], encodedDataTrain["X"][1], encodedDataTrain["X"][2], encodedDataTrain["X"][3], encodedDataTrain["X"][4], encodedDataTrain["X"][5], encodedDataTrain["X"][6], encodedDataTrain["X"][7], encodedDataTrain["X"][8], encodedDataTrain["X"][9], encodedDataTrain["X"][10]), axis=1)

In [121]:
XTrain.shape[1]

98

In [122]:
YTrain = np.concatenate((
    encodedDataTrain["Y_kenong"][0], encodedDataTrain["Y_kenong"][1], encodedDataTrain["Y_kenong"][2], encodedDataTrain["Y_kenong"][3], encodedDataTrain["Y_kenong"][4], encodedDataTrain["Y_kenong"][5], encodedDataTrain["Y_kenong"][6], encodedDataTrain["Y_kenong"][7],
    encodedDataTrain["Y_kethuk"][0], encodedDataTrain["Y_kethuk"][1], encodedDataTrain["Y_kethuk"][2], encodedDataTrain["Y_kethuk"][3], encodedDataTrain["Y_kethuk"][4], encodedDataTrain["Y_kethuk"][5], encodedDataTrain["Y_kethuk"][6], encodedDataTrain["Y_kethuk"][7],
    encodedDataTrain["Y_kempul"][0], encodedDataTrain["Y_kempul"][1], encodedDataTrain["Y_kempul"][2], encodedDataTrain["Y_kempul"][3],
    encodedDataTrain["Y_gong"][0], encodedDataTrain["Y_gong"][1],
    encodedDataTrain["Y_kempyang"][0], encodedDataTrain["Y_kempyang"][1], encodedDataTrain["Y_kempyang"][2], encodedDataTrain["Y_kempyang"][3], encodedDataTrain["Y_kempyang"][4], encodedDataTrain["Y_kempyang"][5], encodedDataTrain["Y_kempyang"][6], encodedDataTrain["Y_kempyang"][7],
    encodedDataTrain["Y_bb"][0], encodedDataTrain["Y_bb"][1], encodedDataTrain["Y_bb"][2], encodedDataTrain["Y_bb"][3], encodedDataTrain["Y_bb"][4], encodedDataTrain["Y_bb"][5], encodedDataTrain["Y_bb"][6], encodedDataTrain["Y_bb"][7], encodedDataTrain["Y_bb"][8], encodedDataTrain["Y_bb"][9], encodedDataTrain["Y_bb"][10], encodedDataTrain["Y_bb"][11], encodedDataTrain["Y_bb"][12], encodedDataTrain["Y_bb"][13], encodedDataTrain["Y_bb"][14], encodedDataTrain["Y_bb"][15], encodedDataTrain["Y_bb"][16],
    encodedDataTrain["Y_bb"][17], encodedDataTrain["Y_bb"][18], encodedDataTrain["Y_bb"][19], encodedDataTrain["Y_bb"][20], encodedDataTrain["Y_bb"][21], encodedDataTrain["Y_bb"][22], encodedDataTrain["Y_bb"][23], encodedDataTrain["Y_bb"][24], encodedDataTrain["Y_bb"][25], encodedDataTrain["Y_bb"][26], encodedDataTrain["Y_bb"][27], encodedDataTrain["Y_bb"][28], encodedDataTrain["Y_bb"][29], encodedDataTrain["Y_bb"][30], encodedDataTrain["Y_bb"][31],
    encodedDataTrain["Y_bp"][0], encodedDataTrain["Y_bp"][1], encodedDataTrain["Y_bp"][2], encodedDataTrain["Y_bp"][3], encodedDataTrain["Y_bp"][4], encodedDataTrain["Y_bp"][5], encodedDataTrain["Y_bp"][6], encodedDataTrain["Y_bp"][7], encodedDataTrain["Y_bp"][8], encodedDataTrain["Y_bp"][9], encodedDataTrain["Y_bp"][10], encodedDataTrain["Y_bp"][11], encodedDataTrain["Y_bp"][12], encodedDataTrain["Y_bp"][13], encodedDataTrain["Y_bp"][14], encodedDataTrain["Y_bp"][15], encodedDataTrain["Y_bp"][16],
    encodedDataTrain["Y_bp"][17], encodedDataTrain["Y_bp"][18], encodedDataTrain["Y_bp"][19], encodedDataTrain["Y_bp"][20], encodedDataTrain["Y_bp"][21], encodedDataTrain["Y_bp"][22], encodedDataTrain["Y_bp"][23], encodedDataTrain["Y_bp"][24], encodedDataTrain["Y_bp"][25], encodedDataTrain["Y_bp"][26], encodedDataTrain["Y_bp"][27], encodedDataTrain["Y_bp"][28], encodedDataTrain["Y_bp"][29], encodedDataTrain["Y_bp"][30], encodedDataTrain["Y_bp"][31], encodedDataTrain["Y_bp"][32],
    encodedDataTrain["Y_bp"][33], encodedDataTrain["Y_bp"][34], encodedDataTrain["Y_bp"][35], encodedDataTrain["Y_bp"][36], encodedDataTrain["Y_bp"][37], encodedDataTrain["Y_bp"][38], encodedDataTrain["Y_bp"][39], encodedDataTrain["Y_bp"][40], encodedDataTrain["Y_bp"][41], encodedDataTrain["Y_bp"][42], encodedDataTrain["Y_bp"][43], encodedDataTrain["Y_bp"][44], encodedDataTrain["Y_bp"][45], encodedDataTrain["Y_bp"][46], encodedDataTrain["Y_bp"][47], encodedDataTrain["Y_bp"][48],
    encodedDataTrain["Y_bp"][49], encodedDataTrain["Y_bp"][50], encodedDataTrain["Y_bp"][51], encodedDataTrain["Y_bp"][52], encodedDataTrain["Y_bp"][53], encodedDataTrain["Y_bp"][54], encodedDataTrain["Y_bp"][55], encodedDataTrain["Y_bp"][56], encodedDataTrain["Y_bp"][57], encodedDataTrain["Y_bp"][58], encodedDataTrain["Y_bp"][59], encodedDataTrain["Y_bp"][60], encodedDataTrain["Y_bp"][61], encodedDataTrain["Y_bp"][62], encodedDataTrain["Y_bp"][63],
    encodedDataTrain["Y_peking"][0], encodedDataTrain["Y_peking"][1], encodedDataTrain["Y_peking"][2], encodedDataTrain["Y_peking"][3], encodedDataTrain["Y_peking"][4], encodedDataTrain["Y_peking"][5], encodedDataTrain["Y_peking"][6], encodedDataTrain["Y_peking"][7], encodedDataTrain["Y_peking"][8], encodedDataTrain["Y_peking"][9], encodedDataTrain["Y_peking"][10], encodedDataTrain["Y_peking"][11], encodedDataTrain["Y_peking"][12], encodedDataTrain["Y_peking"][13], encodedDataTrain["Y_peking"][14], encodedDataTrain["Y_peking"][15], encodedDataTrain["Y_peking"][16],
    encodedDataTrain["Y_peking"][17], encodedDataTrain["Y_peking"][18], encodedDataTrain["Y_peking"][19], encodedDataTrain["Y_peking"][20], encodedDataTrain["Y_peking"][21], encodedDataTrain["Y_peking"][22], encodedDataTrain["Y_peking"][23], encodedDataTrain["Y_peking"][24], encodedDataTrain["Y_peking"][25], encodedDataTrain["Y_peking"][26], encodedDataTrain["Y_peking"][27], encodedDataTrain["Y_peking"][28], encodedDataTrain["Y_peking"][29], encodedDataTrain["Y_peking"][30], encodedDataTrain["Y_peking"][31]
), axis=1)

In [123]:
YTrain.shape[1]

1216

In [124]:
latent_dim = 98
conditional_dim = XTrain.shape[1]
output_dim = YTrain.shape[1]

generator = buildGenerator(latent_dim, conditional_dim, output_dim)
discriminator = buildDiscriminator(output_dim)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
discriminator.trainable = True
gan = buildGan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [125]:
generator.summary()

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15            │ (None, 98)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_16            │ (None, 98)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 196)            │              0 │ input_layer_15[0][0],  │
│ (Concatenate)             │                        │                │ input_layer_16[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_21 (Dense)          │ (None, 128)            │         25,216 │ concatenate_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_15            │ (None, 128)            │              0 │ dense_21[0][0]         │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_22 (Dense)          │ (None, 256)            │         33,024 │ leaky_re_lu_15[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_16            │ (None, 256)            │              0 │ dense_22[0][0]         │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_23 (Dense)          │ (None, 512)            │        131,584 │ leaky_re_lu_16[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_17            │ (None, 512)            │              0 │ dense_23[0][0]         │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_24 (Dense)          │ (None, 1216)           │        623,808 │ leaky_re_lu_17[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 813,632 (3.10 MB)

 Trainable params: 813,632 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

In [126]:
discriminator.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_25 (Dense)                     │ (None, 512)                 │         623,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_18 (LeakyReLU)           │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_19 (LeakyReLU)           │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 754,689 (2.88 MB)

 Trainable params: 754,689 (2.88 MB)

 Non-trainable params: 0 (0.00 B)

In [127]:
gan.summary()

Model: "functional_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18            │ (None, 98)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_19            │ (None, 98)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_27             │ (None, 1216)           │        813,632 │ input_layer_18[0][0],  │
│ (Functional)              │                        │                │ input_layer_19[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_3 (Sequential) │ (None, 1)              │        754,689 │ functional_27[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,568,321 (5.98 MB)

 Trainable params: 1,568,321 (5.98 MB)

 Non-trainable params: 0 (0.00 B)

In [128]:
conditional_input = np.expand_dims(XTrain[0], axis=0)
latent_vector = np.random.normal(0, 1, (1, latent_dim))
# latent_vector = np.expand_dims(XTrain[0], axis=0)
generated_output = generator.predict([latent_vector, conditional_input])

print("Generated Output:", generated_output)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Generated Output: [[ 0.02943138 -0.09089895  0.16180581 ...  0.08860099  0.05890984
  -0.39814252]]


In [129]:
indeksInstrumenX = {
    "X": 11
}

indeksInstrumenY = {
    "Y_kenong": 8,
    "Y_kethuk": 8,
    "Y_kempul": 4,
    "Y_gong": 2,
    "Y_kempyang": 8,
    "Y_bb": 32,
    "Y_bp": 64,
    "Y_peking": 32
}

In [130]:
print(XTrain[0].shape)

(98,)


In [131]:
print(conditional_input.shape)

(1, 98)


In [132]:
def showNotes(decodedDict, judul):
    print(f"{'='*50} {judul} {'='*50}")
    for key, value in decodedDict.items():
        print(f"{key}: {value}")

In [133]:
decodedInputX = decodeData(latent_vector[0], encodersTrain, indeksInstrumenX)
decodedOutputY = decodeData(generated_output[0], encodersTrain, indeksInstrumenY)
decodedInputRealDataX = decodeData(XTrain, encodersTrain, indeksInstrumenX, indexAt=0)
decodedOutputRealDataY = decodeData(YTrain, encodersTrain, indeksInstrumenY, indexAt=0)

In [134]:
showNotes(decodedInputX, "Latent Vector")
showNotes(decodedOutputY, "Data Generator")
print("\nData Asli:\n")
showNotes(decodedInputRealDataX, "Data Asli X")
showNotes(decodedOutputRealDataY, "Data Asli Y")

================================================== Latent Vector ==================================================
X: [1, 2, 0, 6, 'lagu', 1, 2, '2', '5', '3', "3'"]
================================================== Data Generator ==================================================
Y_kenong: [2, 5, 5, 6, 1, 6, 5, 2]
Y_kethuk: [1, 0, 1, 0, 1, 0, 0, 1]
Y_kempul: [3, 0, 2, 7]
Y_gong: [5, 1]
Y_kempyang: [1, 0, 1, 1, 1, 1, 1, 0]
Y_bb: ['2_', '6', '1', '0', "2'_", '6', '6', '0', '3', '6.0', '2_', 'nan', "2'", '0.0', '2', '0.0', '2', '6.0', "5'", 'nan', "2'_", '0.0', "1'", 'nan', '0', '0', '1_', '0.0', '7', 'nan', '7', '5']
Y_bp: ['3', '0', '0', '0', '0', '0', '2', '0', '1_', '0', "6'", '0', '7', '0', '2', '0', '0', '0', "1'", '0', '6_', '0', '0', '0', '4', '0', "2'_", '0', "2'", '0', "5'", '0', '2', '0', '7', '0', '6', '0', '5', '0', "1'", '0', '3', '0', "1'", '0', '0', '0', '2_', '0', '3', '0.0', '2_', '0', '7', '0', '3_', '0', '6', '0', "1'", '0', '3', '0']
Y_peking: ['6,', '0', '6', '0',

In [136]:
print(

SyntaxError: incomplete input (<ipython-input-136-424fbb3a34c5>, line 1)

In [137]:
# target asli dari YTrain[0]
actual_output = YTrain[0]
mse = mean_squared_error(actual_output, generated_output.flatten())
print("Mean Squared Error antara aktual dan generat output:", mse)
mae = mean_absolute_error(actual_output, generated_output.flatten())
print("Mean Absolute Error antara aktual dan generat output:", mae)

Mean Squared Error antara aktual dan generat output: 0.13647617400883263
Mean Absolute Error antara aktual dan generat output: 0.2125049280108502


In [138]:
print(conditional_input)
print(latent_vector)

[[1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0.]]
[[-7.54022311e-01  2.43359744e-01  4.58717746e-02 -1.03988311e+00
  -7.02211983e-01 -2.40734595e+00  1.15512739e+00  5.10061583e-01
  -2.90716444e-01  3.92071741e-01  2.24722153e-01  5.81141380e-02
  -1.38257639e+00  5.55087661e-01  7.15327115e-01 -2.99427561e+00
   5.43864975e-02  1.50610259e+00  2.36489050e+00  4.51963791e-01
  -1.04700607e+00 -1.57109495e+00  1.94936815e+00  1.01078660e-01
  -1.27871577e+00  7.48201664e-01 -3.29110830e-02 -4.58917940e-01
  -5.63184533e-01 -1.76276000e-01 -3.76193679e-01 -1.49094890e-02
   1.05530367e+00  9.26621071e-01  7.18794907e-01  8.44471914e-01
   4.47275962e-01  1.09172534e+00  5.80843939e-01 -4.47330484e-01
  -1.01118792e+00  9.10702571e-02 

In [139]:
print(

SyntaxError: incomplete input (<ipython-input-139-424fbb3a34c5>, line 1)

In [ ]:
generator.save('generator.h5')
discriminator.save('discriminator.h5')
gan.save("gan.h5")

In [ ]:
# Latih GAN
d_losses_real, d_losses_fake, g_losses = train_gan(gan, generator, discriminator, XTrain, YTrain, latent_dim)

In [ ]:
# plt.plot(d_losses_real, label='Discriminator loss real')
plt.plot(d_losses_fake, label='Discrimator loss fake')
plt.plot(g_losses, label="Generator losses")
plt.legend()
plt.show()

# d_losses_real, d_losses_fake, g_losses

In [ ]:
generatorLoaded = load_model("generator.h5")
discriminatorLoaded = load_model("discriminator.h5")
ganLoaded = load_model("gan.h5")

# Data Test

In [140]:
XTest = np.concatenate((encodedDataTest["X"][0], encodedDataTest["X"][1], encodedDataTest["X"][2], encodedDataTest["X"][3], encodedDataTest["X"][4], encodedDataTest["X"][5], encodedDataTest["X"][6], encodedDataTest["X"][7], encodedDataTest["X"][8], encodedDataTest["X"][9], encodedDataTest["X"][10]), axis=1)

In [141]:
XTest.shape[1]

75

In [142]:
YTest = np.concatenate((
    encodedDataTest["Y_kenong"][0], encodedDataTest["Y_kenong"][1], encodedDataTest["Y_kenong"][2], encodedDataTest["Y_kenong"][3], encodedDataTest["Y_kenong"][4], encodedDataTest["Y_kenong"][5], encodedDataTest["Y_kenong"][6], encodedDataTest["Y_kenong"][7],
    encodedDataTest["Y_kethuk"][0], encodedDataTest["Y_kethuk"][1], encodedDataTest["Y_kethuk"][2], encodedDataTest["Y_kethuk"][3], encodedDataTest["Y_kethuk"][4], encodedDataTest["Y_kethuk"][5], encodedDataTest["Y_kethuk"][6], encodedDataTest["Y_kethuk"][7],
    encodedDataTest["Y_kempul"][0], encodedDataTest["Y_kempul"][1], encodedDataTest["Y_kempul"][2], encodedDataTest["Y_kempul"][3],
    encodedDataTest["Y_gong"][0], encodedDataTest["Y_gong"][1],
    encodedDataTest["Y_kempyang"][0], encodedDataTest["Y_kempyang"][1], encodedDataTest["Y_kempyang"][2], encodedDataTest["Y_kempyang"][3], encodedDataTest["Y_kempyang"][4], encodedDataTest["Y_kempyang"][5], encodedDataTest["Y_kempyang"][6], encodedDataTest["Y_kempyang"][7],
    encodedDataTest["Y_bb"][0], encodedDataTest["Y_bb"][1], encodedDataTest["Y_bb"][2], encodedDataTest["Y_bb"][3], encodedDataTest["Y_bb"][4], encodedDataTest["Y_bb"][5], encodedDataTest["Y_bb"][6], encodedDataTest["Y_bb"][7], encodedDataTest["Y_bb"][8], encodedDataTest["Y_bb"][9], encodedDataTest["Y_bb"][10], encodedDataTest["Y_bb"][11], encodedDataTest["Y_bb"][12], encodedDataTest["Y_bb"][13], encodedDataTest["Y_bb"][14], encodedDataTest["Y_bb"][15], encodedDataTest["Y_bb"][16],
    encodedDataTest["Y_bb"][17], encodedDataTest["Y_bb"][18], encodedDataTest["Y_bb"][19], encodedDataTest["Y_bb"][20], encodedDataTest["Y_bb"][21], encodedDataTest["Y_bb"][22], encodedDataTest["Y_bb"][23], encodedDataTest["Y_bb"][24], encodedDataTest["Y_bb"][25], encodedDataTest["Y_bb"][26], encodedDataTest["Y_bb"][27], encodedDataTest["Y_bb"][28], encodedDataTest["Y_bb"][29], encodedDataTest["Y_bb"][30], encodedDataTest["Y_bb"][31],
    encodedDataTest["Y_bp"][0], encodedDataTest["Y_bp"][1], encodedDataTest["Y_bp"][2], encodedDataTest["Y_bp"][3], encodedDataTest["Y_bp"][4], encodedDataTest["Y_bp"][5], encodedDataTest["Y_bp"][6], encodedDataTest["Y_bp"][7], encodedDataTest["Y_bp"][8], encodedDataTest["Y_bp"][9], encodedDataTest["Y_bp"][10], encodedDataTest["Y_bp"][11], encodedDataTest["Y_bp"][12], encodedDataTest["Y_bp"][13], encodedDataTest["Y_bp"][14], encodedDataTest["Y_bp"][15], encodedDataTest["Y_bp"][16],
    encodedDataTest["Y_bp"][17], encodedDataTest["Y_bp"][18], encodedDataTest["Y_bp"][19], encodedDataTest["Y_bp"][20], encodedDataTest["Y_bp"][21], encodedDataTest["Y_bp"][22], encodedDataTest["Y_bp"][23], encodedDataTest["Y_bp"][24], encodedDataTest["Y_bp"][25], encodedDataTest["Y_bp"][26], encodedDataTest["Y_bp"][27], encodedDataTest["Y_bp"][28], encodedDataTest["Y_bp"][29], encodedDataTest["Y_bp"][30], encodedDataTest["Y_bp"][31], encodedDataTest["Y_bp"][32],
    encodedDataTest["Y_bp"][33], encodedDataTest["Y_bp"][34], encodedDataTest["Y_bp"][35], encodedDataTest["Y_bp"][36], encodedDataTest["Y_bp"][37], encodedDataTest["Y_bp"][38], encodedDataTest["Y_bp"][39], encodedDataTest["Y_bp"][40], encodedDataTest["Y_bp"][41], encodedDataTest["Y_bp"][42], encodedDataTest["Y_bp"][43], encodedDataTest["Y_bp"][44], encodedDataTest["Y_bp"][45], encodedDataTest["Y_bp"][46], encodedDataTest["Y_bp"][47], encodedDataTest["Y_bp"][48],
    encodedDataTest["Y_bp"][49], encodedDataTest["Y_bp"][50], encodedDataTest["Y_bp"][51], encodedDataTest["Y_bp"][52], encodedDataTest["Y_bp"][53], encodedDataTest["Y_bp"][54], encodedDataTest["Y_bp"][55], encodedDataTest["Y_bp"][56], encodedDataTest["Y_bp"][57], encodedDataTest["Y_bp"][58], encodedDataTest["Y_bp"][59], encodedDataTest["Y_bp"][60], encodedDataTest["Y_bp"][61], encodedDataTest["Y_bp"][62], encodedDataTest["Y_bp"][63],
    encodedDataTest["Y_peking"][0], encodedDataTest["Y_peking"][1], encodedDataTest["Y_peking"][2], encodedDataTest["Y_peking"][3], encodedDataTest["Y_peking"][4], encodedDataTest["Y_peking"][5], encodedDataTest["Y_peking"][6], encodedDataTest["Y_peking"][7], encodedDataTest["Y_peking"][8], encodedDataTest["Y_peking"][9], encodedDataTest["Y_peking"][10], encodedDataTest["Y_peking"][11], encodedDataTest["Y_peking"][12], encodedDataTest["Y_peking"][13], encodedDataTest["Y_peking"][14], encodedDataTest["Y_peking"][15], encodedDataTest["Y_peking"][16],
    encodedDataTest["Y_peking"][17], encodedDataTest["Y_peking"][18], encodedDataTest["Y_peking"][19], encodedDataTest["Y_peking"][20], encodedDataTest["Y_peking"][21], encodedDataTest["Y_peking"][22], encodedDataTest["Y_peking"][23], encodedDataTest["Y_peking"][24], encodedDataTest["Y_peking"][25], encodedDataTest["Y_peking"][26], encodedDataTest["Y_peking"][27], encodedDataTest["Y_peking"][28], encodedDataTest["Y_peking"][29], encodedDataTest["Y_peking"][30], encodedDataTest["Y_peking"][31]
), axis=1)

In [143]:
YTest.shape[1]

728

In [144]:
latent_dim = 75
conditional_dim = XTest.shape[1]
output_dim = YTest.shape[1]

generator = buildGenerator(latent_dim, conditional_dim, output_dim)
discriminator = buildDiscriminator(output_dim)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
discriminator.trainable = True

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [145]:
generator.summary()

Model: "functional_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20            │ (None, 75)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_21            │ (None, 75)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 150)            │              0 │ input_layer_20[0][0],  │
│ (Concatenate)             │                        │                │ input_layer_21[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_28 (Dense)          │ (None, 128)            │         19,328 │ concatenate_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_20            │ (None, 128)            │              0 │ dense_28[0][0]         │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_29 (Dense)          │ (None, 256)            │         33,024 │ leaky_re_lu_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_21            │ (None, 256)            │              0 │ dense_29[0][0]         │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_30 (Dense)          │ (None, 512)            │        131,584 │ leaky_re_lu_21[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_22            │ (None, 512)            │              0 │ dense_30[0][0]         │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_31 (Dense)          │ (None, 728)            │        373,464 │ leaky_re_lu_22[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 557,400 (2.13 MB)

 Trainable params: 557,400 (2.13 MB)

 Non-trainable params: 0 (0.00 B)

In [146]:
discriminator.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                     │ (None, 512)                 │         373,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_23 (LeakyReLU)           │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_24 (LeakyReLU)           │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 504,833 (1.93 MB)

 Trainable params: 504,833 (1.93 MB)

 Non-trainable params: 0 (0.00 B)

In [147]:
gan = buildGan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

In [148]:
conditional_input = np.expand_dims(XTest[0], axis=0)
latent_vector = np.random.normal(0, 1, (1, latent_dim))
generated_output = generator.predict([latent_vector, conditional_input])

print("Generated Output:", generated_output[:,:20])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Generated Output: [[ 0.05152722  0.03647455 -0.03631733  0.0354719   0.0328705   0.00673495
  -0.05764304 -0.10305313  0.06777994 -0.44694117 -0.00714916 -0.17658547
   0.04926841 -0.29351205 -0.13206577 -0.14526206 -0.00246806 -0.0755135
  -0.07060181 -0.13798474]]


In [149]:
decodedInputX = decodeData(latent_vector[0], encodersTest, indeksInstrumenX)
decodedOutputY = decodeData(generated_output[0], encodersTest, indeksInstrumenY)
decodedInputRealDataX = decodeData(XTest, encodersTest, indeksInstrumenX, indexAt=0)
decodedOutputRealDataY = decodeData(YTest, encodersTest, indeksInstrumenY, indexAt=0)

In [150]:
showNotes(decodedInputX, "Latent Vector")
showNotes(decodedOutputY, "Data Generator")
print("\nData Asli:\n")
showNotes(decodedInputRealDataX, "Data Asli X")
showNotes(decodedOutputRealDataY, "Data Asli Y")

================================================== Latent Vector ==================================================
X: [0, 1, 1, 0, 'lagu', 11, 4, '3', '7,', "2'", '4']
================================================== Data Generator ==================================================
Y_kenong: [0, 3, 2, 1, 3, 0, 2, 0]
Y_kethuk: [0, 0, 0, 0, 0, 0, 1, 0]
Y_kempul: [2, 0, 3, 5]
Y_gong: [7, 1]
Y_kempyang: [1, 0, 0, 0, 1, 0, 0, 0]
Y_bb: ["6'", '0', "5'", '0', '5', '0', '0', '0', '35_', '0', "5'", '0', "6'", '0', '3', '0', '7_', '0', '6', '0', '3_', '0', "1'", '0', "5'", '0', '6', '0', "5'", '0', "3'", 'nan']
Y_bp: ["1'", '0', "5'", '0', "1'", '0', '0', '0', '0', '0', '5_', '0', '0', '0', '0', '0', "5'", '0', '0', '0', '5_', '0', '0', '0', "6'", '0', '3_', '0', "1'", '0', "1'", '0', "1'", '0', '0', '0', '3', '0', '0', '0', "5'", '0', "2'", '0', "3'", '0', '0', '0', '6', '0', '6', '0', "3'", '0', '0', '0', "2'", '0', '6', '0', "6'", '0', '6', '0']
Y_peking: ['5', '0', '3', '0', '6', '0', '5'

In [151]:
# target asli dari Y[0]
actual_output = YTest[0]
mse = mean_squared_error(actual_output, generated_output.flatten())
print("Mean Squared Error antara aktual dan generat output:", mse)
mae = mean_absolute_error(actual_output, generated_output.flatten())
print("Mean Absolute Error antara aktual dan generat output:", mae)

Mean Squared Error antara aktual dan generat output: 0.2571568608000015
Mean Absolute Error antara aktual dan generat output: 0.33430297280782334
